## Assignment 4 (50 marks)
#### =====================================================================================================
### Deadline: 11/02 11:59 pm
#### =====================================================================================================

### Problem 1: PCA (20 marks)

`lab04_dataset_1.csv` contains 205 observations on various vehicles. This is an unsupervised training data. You will use the entire dataset for `PCA`.

### 1.a (2 marks)

For the 14 input features, drop any rows with missing numerical values and output the new length of the training dataset.

In [18]:
import pandas as pd

def process_data(file_path):
    df = pd.read_csv(file_path)
    df.replace('?', pd.NA, inplace=True)
    df.dropna(inplace=True)
    df.drop(columns='price')
    return df

if __name__ == "__main__":
    cleaned_data = process_data("lab04_dataset_1.csv")
    print(cleaned_data.shape)

(195, 14)


### 1.b (6 marks)

Using the sklearn's [`PCA`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) method, compute all the principal components (PCs) of the normalized dataset. All the PCs capture a fraction of the total variance, output all the variances captured by all the PCs. Write a code snippet that checks all the PCs and selects the `top k PCs` whose total variance captured is atleast `90%`. What did `k` come out to be?

In [19]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA 

def run_pca(path="lab04_dataset_1.csv", threshold=0.9, verbose=True):
    X = process_data(path)
    Xn = StandardScaler().fit_transform(X)
    pca = PCA(n_components=None)
    pca.fit(Xn)
    ratios = pca.explained_variance_ratio_
    cumalative_sum = np.cumsum(ratios)
    k = np.searchsorted(cumalative_sum, threshold) + 1
    
    if verbose:
        print("Explained variance ratio per PC:")
        for i, r in enumerate(ratios, 1):
            print(f"PC{i:02d}: {r:.6f}")
        print(f"\nCumulative variance at k={k}: {cumalative_sum[k-1]:.4f}")   
        print(f"\nk = {k} PCs to reach at least 90% total variance") 
    return Xn, k
    
if __name__ == "__main__":
    run_pca()

Explained variance ratio per PC:
PC01: 0.537987
PC02: 0.162802
PC03: 0.086867
PC04: 0.064973
PC05: 0.043496
PC06: 0.029693
PC07: 0.022900
PC08: 0.019296
PC09: 0.008594
PC10: 0.007900
PC11: 0.005828
PC12: 0.004587
PC13: 0.003671
PC14: 0.001407

Cumulative variance at k=6: 0.9258

k = 6 PCs to reach at least 90% total variance


### 1.c (3 marks)

Using the `top k PCs`, apply `dimensionality reduction` on the normalized dataset to generate and display the transformed dataset which should now have only `k columns`. Display the output as a DataFrame.

In [20]:
def dim_reduction(path="lab04_dataset_1.csv"):
    Xn, k = run_pca(path, 0.9, False)
    
    X_reduced = PCA(n_components=k).fit(Xn).transform(Xn)[:, :k]
    reduced_df = pd.DataFrame(X_reduced, columns=[f"PC{i+1}" for i in range(k)])
    print(reduced_df.head())

if __name__ == "__main__":
    dim_reduction()

        PC1       PC2       PC3       PC4       PC5       PC6
0 -0.612999 -2.164573 -0.298752 -2.436486  0.194563 -0.116741
1 -0.493886 -2.190732 -0.248341 -2.476630  0.331332 -0.157826
2  0.443900 -1.365449  1.449443  0.626999  0.359939 -2.002387
3 -0.178982 -0.256392  0.066248  1.153280  0.277470  0.118314
4  1.269804 -1.167075  0.018756  1.204362  0.048029 -0.332126


### 1.d (3 marks)

We learned in class that we can also obtain the PCs using a matrix decomposition technique called `SVD:` $X=U\Lambda V$. Use `SVD` on the original normalized dataset to obtain the 3 decomposed matrices and output them.

In [25]:
def run_svd(path="lab04_dataset_1.csv"):
    X = process_data(path)
    Xn = StandardScaler().fit_transform(X)

    U, s, V = np.linalg.svd(Xn, full_matrices=False)
    Sigma = np.diag(s)
    
    U_df = pd.DataFrame(U, X.index, columns=[f"u{i+1}" for i in range(U.shape[1])])
    Sigma_df = pd.DataFrame(Sigma, columns=[f"s{i+1}" for i in range(Sigma.shape[1])])
    V_df = pd.DataFrame(V, index=[f"PC{i+1:02d}" for i in range(V.shape[0])])
    
    print("U", U_df.head(), "\n")
    print("Sigma", Sigma_df.head(), "\n")
    print("V", V_df.head())
    
    return U_df, Sigma_df, V_df
    
if __name__ == "__main__":
    run_svd()

U          u1        u2        u3        u4        u5        u6        u7  \
0 -0.015995  0.102674  0.019400  0.182944 -0.017855  0.012966  0.063217   
1 -0.012887  0.103915  0.016127  0.185958 -0.030406  0.017530  0.047363   
2  0.011583  0.064769 -0.094122 -0.047078 -0.033031  0.222402 -0.091994   
3 -0.004670  0.012162 -0.004302 -0.086594 -0.025463 -0.013141  0.022138   
4  0.033134  0.055359 -0.001218 -0.090430 -0.004408  0.036889 -0.016175   

         u8        u9       u10       u11       u12       u13       u14  
0  0.134816  0.031601  0.044295 -0.111749  0.101114 -0.048439  0.008628  
1  0.121088  0.010908  0.079402 -0.154791  0.079009 -0.062272  0.021627  
2  0.183118  0.041210 -0.102957  0.007346 -0.098261 -0.027142 -0.067653  
3  0.012599 -0.033787  0.019578 -0.072130 -0.066190  0.082089  0.012842  
4  0.097954 -0.052015  0.089414 -0.012797  0.044259 -0.032666  0.035416   

Sigma           s1         s2         s3         s4         s5   s6   s7   s8   s9  \
0  38.323675   

### 1.e (3 marks)

Generate the `k` (obtained from `1.b`) largest eigenvalues from the decomposed matrices obtained from the `SVD`. Remember, eigenvalue $\lambda=\Lambda^2/n$

In [34]:
def largest_eigen(path="lab04_dataset_1.csv"):
    Xn, k = run_pca(path, verbose=False)
    
    U, s, V = np.linalg.svd(Xn, full_matrices=False)

    eigvals = (s**2) / Xn.shape[0]
    return eigvals[:k]

if __name__ == "__main__":
    print(largest_eigen())

[7.53181553 2.27923094 1.21613308 0.90961519 0.60894217 0.4157043 ]


### 1.f (3 marks)

Generate the projections of the normalized dataset using the `first k PCs` obtained from the `SVD` and display it inside a DataFrame.

In [ ]:
def projection():
    Xn, k = run_pca(verbose=False)
    U, s, v = np.linalg.svd(Xn, full_matrices=False)
    
    Z = U[:, :k] * s[:k]
    proj_df = pd.DataFrame(Z, columns=[f"PC{i+1}" for i in range(k)])
    print(proj_df.head())
    
if __name__ == "__main__":
    projection()

### Problem 2: Clustering (30 marks)

`lab04_dataset_2.csv` contains 239 observations with two input features *x1* and *x2*.

`lab04_dataset_3.csv` contains 1440 observations with two input features *x1* and *x2*.

For this task, you will perform various clustering-related operations using sklearn's [`clustering`](https://scikit-learn.org/stable/modules/clustering.html) module.

### 2.a (6 marks)

Using `lab04_dataset_2.csv`, apply sklearn's `KMeans` algorithm on the two-dimensional data and output the resulting clusters using a scatterplot. You will apply `KMeans` over several clusters ranging from cluster-count `K = 2 to 6`. Make sure for every iteration of different cluster-count, your scatterplot should use `K colors` to clearly distinguish the data points belonging in their respective `K clusters`. Also, compute the `Silhouette score` for each of those `K clusters` and plot that score against `K`. Label the plot axes accordingly.

### 2.b (6 marks)

Repeat `2.a` but instead use sklearn's `GaussianMixture` model for learning.

### 2.c (6 marks)

Using `lab04_dataset_3.csv`, apply sklearn's `AgglomerativeClustering` on the two-dimensional data and output the resulting clusters using a scatterplot. You will apply `AgglomerativeClustering` over several clusters ranging from cluster-count `K = 2 to 6`. Make sure your scatterplot uses `K colors` to clearly distinguish the data points belonging in their respective `K clusters`. Also, compute the `Silhouette score` for each of those `K clusters` and plot the `Silhouette score` against `K clusters`. Label the plot axes accordingly.

### 2.d (6 marks)

Repeat `2.c` but instead use sklearn's `SpectralClustering` model for learning.

### 2.e (6 marks)

The dataset `lab04_dataset_3.csv` generates 4 concentric rings, so ideally we would want 4 clusters representing the 4 concentric rings. Did the clustering attempts in `2.c` and `2.d` lead to 4 concentric ring clusters. Explore some other sklearn clustering algorithms to see which one can actually produce 4 clusters corresponding with the 4 concentric rings and display it.